In [7]:
import torch
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from ipynb.fs.defs.train import CharCNN
import os

In [8]:
train_dir = os.getcwd() + "/../train"
train_data_dir = os.getcwd() + "/../train_chars"
val_dir = os.getcwd() + "/../test"
val_data_dir = "/../test_chars"
model_weights_path = os.path.join(os.getcwd(), '../model/char_cnn.pth')

In [ ]:
device = torch.device("cuda")
model = CharCNN(num_classes=36).to(device)
model.load_state_dict(torch.load(model_weights_path, map_location=device))  
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for imgs, labels in val_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)